In [62]:
import pandas as pd
import datetime
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from keras import optimizers
from tensorflow import keras
import keras_tuner
from tensorflow.keras.callbacks import EarlyStopping
from  tensorflow.keras.metrics import mean_squared_error
import numpy as np

In [63]:
df_master = pd.read_csv("../../datasources/master_dataset.csv")
df_master.head()

,date,rate,sent_fin_us_uncertainty,sent_fin_us_litigious,sent_fin_us_strong_modal,sent_fin_us_weak_modal,sent_fin_us_constraining,sent_fin_us_optimistic,sent_fin_can_uncertainty,sent_fin_can_litigious,...,index_WGTS.FOPR,index_WGTS.MTLS,index_WGTS.NATURALGAS,index_WGTS.WCC,index_WGTS.WTI,us_bank_rate,sent_bert_us_fin,sent_bert_us_gen,sent_bert_can_fin_sentiment,sent_bert_can_gen_sentiment
0,1955-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.25,NaN,NaN,NaN,NaN
1,1955-08-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.25,NaN,NaN,NaN,NaN
2,1955-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.25,NaN,NaN,NaN,NaN
3,1955-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.25,NaN,NaN,NaN,NaN
4,1955-08-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.25,NaN,NaN,NaN,NaN


### Set Label for each row


In [64]:
df_master['y_exp'] = np.full(df_master.shape[0], -1)

for i, row in df_master.iterrows():
    try:
        two_weeks_from_current_date = datetime.datetime.strptime(row.date, "%Y-%m-%d") + datetime.timedelta(weeks=2)

        futur_rate = list(df_master.loc[df_master['date'] == str(two_weeks_from_current_date.date())].rate)[0]
        df_master.loc[i, 'y_exp' ] = futur_rate
    except: 
        break

df_master = df_master.loc[df_master.y_exp >= 0]


In [65]:
df_master= df_master.dropna()
df_master= df_master.reset_index(drop=True)

In [66]:
df_master

,date,rate,sent_fin_us_uncertainty,sent_fin_us_litigious,sent_fin_us_strong_modal,sent_fin_us_weak_modal,sent_fin_us_constraining,sent_fin_us_optimistic,sent_fin_can_uncertainty,sent_fin_can_litigious,...,index_WGTS.MTLS,index_WGTS.NATURALGAS,index_WGTS.WCC,index_WGTS.WTI,us_bank_rate,sent_bert_us_fin,sent_bert_us_gen,sent_bert_can_fin_sentiment,sent_bert_can_gen_sentiment,y_exp
0,1995-05-15,1.3515,32.0,17.0,2.0,10.0,11.0,-35.0,47.0,6.0,...,19.202994,7.496358,5.324944,11.210801,9.00,0.336596,0.86182,0.441466,0.866923,1.3720
1,1995-05-16,1.3590,32.0,17.0,2.0,10.0,11.0,-35.0,47.0,6.0,...,19.202994,7.496358,5.324944,11.210801,9.00,0.336596,0.86182,0.441466,0.866923,1.3693
2,1995-05-17,1.3557,32.0,17.0,2.0,10.0,11.0,-35.0,47.0,6.0,...,19.202994,7.496358,5.324944,11.210801,9.00,0.336596,0.86182,0.441466,0.866923,1.3717
3,1995-05-18,1.3555,32.0,17.0,2.0,10.0,11.0,-35.0,47.0,6.0,...,19.202994,7.496358,5.324944,11.210801,9.00,0.336596,0.86182,0.441466,0.866923,1.3705
4,1995-05-19,1.3551,32.0,17.0,2.0,10.0,11.0,-35.0,47.0,6.0,...,19.202994,7.496358,5.324944,11.210801,9.00,0.336596,0.86182,0.441466,0.866923,1.3740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7143,2022-10-17,1.3877,13.0,9.0,2.0,7.0,5.0,-2.0,40.0,3.0,...,17.131522,6.758784,24.702028,26.021942,6.25,0.514009,0.97971,0.316318,0.703847,1.3615
7144,2022-10-18,1.3714,13.0,9.0,2.0,7.0,5.0,-2.0,40.0,3.0,...,17.131522,6.758784,24.702028,26.021942,6.25,0.514009,0.97971,0.316318,0.703847,1.3625
7145,2022-10-19,1.3741,13.0,9.0,2.0,7.0,5.0,-2.0,40.0,3.0,...,17.131522,6.758784,24.702028,26.021942,6.25,0.514009,0.97971,0.316318,0.703847,1.3632
7146,2022-10-20,1.3767,20.0,6.0,6.0,12.0,2.0,-26.0,40.0,3.0,...,17.131522,6.758784,24.702028,26.021942,6.25,0.514009,0.97971,0.316318,0.703847,1.3712


In [67]:
def reload_data():
    df = df_master
    df = df.replace('.', np.nan)
    df[[" us_bank_rate"]] = df[[" us_bank_rate"]].apply(pd.to_numeric)
    df = df.rename(columns={' us_bank_rate': 'us_bank_rate'})
    df = df.dropna()
    return df
    
df = reload_data()

## Prepare data for LSTM

In [82]:
def split_train_test_val(df):
    # Split train val test datasets
    test_cutting_date = '2022-05-01'
    val_cutting_date = '2022-02-01'
    df_train = df[df['date'] < val_cutting_date]
    df_val = df[(df['date'] >= val_cutting_date) & (df['date'] < test_cutting_date)]
    df_test = df[df['date'] >= test_cutting_date]
    return df_train, df_val, df_test
df_train, df_val, df_test = split_train_test_val(df)

In [69]:
def split_Xy(df):
    return df.drop(columns=['y_exp']), df['y_exp']

X_train, y_train = split_Xy(df_train)
X_val, y_val = split_Xy(df_val)
X_test, y_test = split_Xy(df_test)

In [70]:
from sklearn.preprocessing import MinMaxScaler

def normalize(df, train=False, scaler=None):
    if train:
        scaler = MinMaxScaler()
    cols_to_normalize = ['sent_fin_us_uncertainty','sent_fin_us_litigious','sent_fin_us_strong_modal','sent_fin_us_weak_modal','sent_fin_us_constraining','sent_fin_us_optimistic','sent_fin_can_uncertainty','sent_fin_can_litigious','sent_fin_can_strong_modal','sent_fin_can_weak_modal','sent_fin_can_constraining','sent_fin_can_optimistic', 'index_W.BCPI','index_W.BCNE','index_W.ENER','index_W.MTLS','index_W.FOPR','index_W.AGRI','index_W.FISH','index_WGTS.AGRI','index_WGTS.BRENT','index_WGTS.COAL','index_WGTS.FISH','index_WGTS.FOPR','index_WGTS.MTLS','index_WGTS.NATURALGAS','index_WGTS.WCC','index_WGTS.WTI']
    if train:
        scaler = scaler.fit(df[cols_to_normalize])
    df[cols_to_normalize] = scaler.transform(df[cols_to_normalize])
    return df, scaler

X_train, scaler = normalize(X_train, train=True)
X_val, _ = normalize(X_val, train=False, scaler=scaler)
X_test, _ = normalize(X_test, train=False, scaler=scaler)


In [71]:
def to_batch(df, y):
    # Put data in batches
    sliding_window_size = 20

    data = []
    for i in range(sliding_window_size, len(df)):
        sample = df.iloc[i-sliding_window_size:i]
        if 'date' in sample:
            sample = sample.drop(columns=['date']) # date is not relevant
        data.append(sample.values)
    data = np.array(data)
    return data, y[sliding_window_size:]

X_train, y_train = to_batch(X_train, y_train)
X_val, y_val = to_batch(X_val, y_val)
X_test, y_test = to_batch(X_test, y_test)

## Train vanila model

In [ ]:


def train_lstm_model(X_train,y_train, X_val,y_val, verbose=2):
    model = Sequential()
    model.add(LSTM(60, input_shape=X_train.shape[1:]))
    model.add(Dense(32))
    model.add(Dense(1))
    optimizer = optimizers.Adam(clipnorm=0.1)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, 
                            verbose=1, mode='auto', restore_best_weights=True)

    info = model.fit(X_train,y_train,validation_data=(X_val,y_val),
            callbacks=[monitor],verbose=verbose,epochs=100)
    return model, info


model, _ = train_lstm_model(X_train,y_train, X_val,y_val)

# Search best features

In [ ]:
from  tensorflow.keras.metrics import mean_squared_error
from tqdm.notebook import tqdm

mses = []
for i in tqdm(range(X_train.shape[-1])):
    x_t = np.expand_dims(X_train[:,:,i], axis=-1)
    x_v = np.expand_dims(X_val[:,:,i], axis=-1)
    x_test = np.expand_dims(X_test[:,:,i], axis=-1)
    model, _ = train_lstm_model(x_t,y_train, x_v,y_val, verbose=0)
    y_pred = model(x_test)
    mses.append(mean_squared_error(y_test.to_numpy(), y_pred.numpy().reshape(-1)))

In [ ]:
indexes = np.argsort(mses)
res = []
for i in np.argsort(mses):
    res.append(df.columns[i+1])
res

**TOP features:** ['interest_rate_can',
 'index_WGTS.BRENT',
 'rate',
 'ti_rate_rsi_14',
 'sent_fin_can_litigious',
 'ti_rate_stochrsi_14',
 'sent_fin_us_strong_modal',
 'ti_rate_williams_%R',
 'index_WGTS.COAL',
 'ti_rate_macd_12_26',
 'sent_fin_us_uncertainty',
 'sent_fin_can_uncertainty',
 'index_W.FISH',
 'ti_rate_atr',
 'sent_fin_us_weak_modal',
 'sent_bert_can_fin_sentiment',
 'index_WGTS.WCC',
 'index_W.ENER',
 'sent_bert_us_gen',
 'sent_bert_can_gen_sentiment',
 'sent_fin_us_litigious',
 'sent_bert_us_fin',
 'index_W.FOPR',
 'ti_rate_utlimate_oscillator',
 'index_WGTS.FOPR',
 'sent_fin_can_strong_modal',
 'ti_rate_adx_14',
 'ti_rate_roc',
 'ti_rate_cci',
 'sent_fin_can_weak_modal',
 'sent_fin_can_optimistic',
 'sent_fin_us_optimistic',
 'index_WGTS.NATURALGAS',
 'sent_fin_can_constraining',
 'us_bank_rate',
 'index_WGTS.MTLS',
 'index_WGTS.AGRI',
 'index_W.MTLS',
 'index_W.BCNE',
 'index_WGTS.WTI',
 'index_WGTS.FISH',
 'sent_fin_us_constraining',
 'index_W.AGRI',
 'index_W.BCPI']

## Select top n features

In [32]:
from sklearn.decomposition import PCA

def do_pca(X, train=True, pca=None):
    if train:
        pca = PCA(n_components='mle', svd_solver='full')
        pca = pca.fit(X)
    return pca.transform(X), pca



In [17]:
cols = ['interest_rate_can',
 'index_WGTS.BRENT',
 'rate',
 'ti_rate_rsi_14',
 'sent_fin_can_litigious',
 'ti_rate_stochrsi_14',
 'sent_fin_us_strong_modal',
 'ti_rate_williams_%R',
 'index_WGTS.COAL',
 'ti_rate_macd_12_26',
 'sent_fin_us_uncertainty',
 'sent_fin_can_uncertainty',
 'index_W.FISH',
 'ti_rate_atr',
 'sent_fin_us_weak_modal',
 'sent_bert_can_fin_sentiment',
 'index_WGTS.WCC',
 'index_W.ENER',
 'sent_bert_us_gen',
 'sent_bert_can_gen_sentiment',
 'sent_fin_us_litigious',
 'sent_bert_us_fin',
 'index_W.FOPR',
 'ti_rate_utlimate_oscillator',
 'index_WGTS.FOPR',
 'sent_fin_can_strong_modal',
 'ti_rate_adx_14',
 'ti_rate_roc',
 'ti_rate_cci',
 'sent_fin_can_weak_modal',
 'sent_fin_can_optimistic',
 'sent_fin_us_optimistic',
 'index_WGTS.NATURALGAS',
 'sent_fin_can_constraining',
 'us_bank_rate',
 'index_WGTS.MTLS',
 'index_WGTS.AGRI',
 'index_W.MTLS',
 'index_W.BCNE',
 'index_WGTS.WTI',
 'index_WGTS.FISH',
 'sent_fin_us_constraining',
 'index_W.AGRI',
 'index_W.BCPI']

In [ ]:
from tqdm.notebook import tqdm
from  tensorflow.keras.metrics import mean_squared_error



mses = []
for i in tqdm(range(0,len(cols), 2)):
    df = reload_data()

    df_train, df_val, df_test = split_train_test_val(df)

    X_train, y_train = split_Xy(df_train)
    X_val, y_val = split_Xy(df_val)
    X_test, y_test = split_Xy(df_test)

    X_train, scaler = normalize(X_train, train=True)
    X_val, _ = normalize(X_val, train=False, scaler=scaler)
    X_test, _ = normalize(X_test, train=False, scaler=scaler)

    X_train, X_val, X_test = X_train[cols[:i+1]], X_val[cols[:i+1]], X_test[cols[:i+1]]

    # X_train, pca = do_pca(X_train, train=True)
    # X_val, _ = do_pca(X_val, train=False, pca=pca)
    # X_test, _ = do_pca(X_test, train=False, pca=pca)



    X_train, y_train = to_batch(X_train, y_train)
    X_val, y_val = to_batch(X_val, y_val)
    X_test, y_test = to_batch(X_test, y_test)

    model, _ = train_lstm_model(X_train,y_train, X_val,y_val)
    y_pred = model(X_test)
    mses.append(mean_squared_error(y_test.to_numpy(), y_pred.numpy().reshape(-1)))

In [ ]:
indexes = np.argsort(mses)
indexes, mses

**Best top n features**: [ 1,  3,  7, 11,  4, 10,  2,  6, 12, 20,  8, 14, 13,  5, 15, 17,  9,
        21,  0, 18, 16, 19]

## Hyperparameter tuning

In [83]:
df = reload_data()

df_train, df_val, df_test = split_train_test_val(df)

X_train, y_train = split_Xy(df_train)
X_val, y_val = split_Xy(df_val)
X_test, y_test = split_Xy(df_test)

X_train, scaler = normalize(X_train, train=True)
X_val, _ = normalize(X_val, train=False, scaler=scaler)
X_test, _ = normalize(X_test, train=False, scaler=scaler)

i =7
X_train, X_val, X_test = X_train[cols[:i+1]], X_val[cols[:i+1]], X_test[cols[:i+1]]

# X_train, pca = do_pca(X_train, train=True)
# X_val, _ = do_pca(X_val, train=False, pca=pca)
# X_test, _ = do_pca(X_test, train=False, pca=pca)

# X_train, X_val, X_test = pd.DataFrame(X_train), pd.DataFrame(X_val), pd.DataFrame(X_test)

X_train, y_train = to_batch(X_train, y_train)
X_val, y_val = to_batch(X_val, y_val)
X_test, y_test = to_batch(X_test, y_test)

# model, _ = train_lstm_model(X_train,y_train, X_val,y_val)
# y_pred = model(X_test)
# mses.append(mean_squared_error(y_test.to_numpy(), y_pred.numpy().reshape(-1)))

In [84]:


def build_model(hp):
    model = Sequential()
    model.add(LSTM(60, input_shape=X_train.shape[1:]))
    for _ in range(hp.Int("units", min_value=1, max_value=3, step=1)):
        model.add(Dense(hp.Int("units", min_value=10, max_value=60, step=10)))
    model.add(Dense(1))
    optimizer = optimizers.Adam(clipnorm=1)
    model.compile(loss='mean_squared_error', optimizer=optimizer)   
    return model


monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, 
                        verbose=1, mode='auto', restore_best_weights=True)

tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective='val_loss',
    max_trials=30,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
    seed=42
)
tuner.search(X_train, y_train, epochs=2, validation_data=(X_val, y_val), callbacks=[monitor])


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
2                 |?                 |units

Epoch 1/2
210/210 [==============================] - 20s 55ms/step - loss: 0.1151 - val_loss: 0.0023
Epoch 2/2
210/210 [==============================] - 6s 27ms/step - loss: 0.0173 - val_loss: 0.0036
Epoch 1/2
210/210 [==============================] - 9s 23ms/step - loss: 0.1941 - val_loss: 0.0043
Epoch 2/2
 55/210 [======>.......................] - ETA: 3s - loss: 0.0134

In [ ]:
# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
# model = build_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
# model.fit(x=x_all, y=y_all, epochs=1)

for i in range(5):
    model = build_model(best_hps[0])
    model.fit(x=x_all, y=y_all, epochs=1)
    model.save(f'model_{i}')


211/211 [==============================] - 13s 30ms/step - loss: 0.0598


INFO:tensorflow:Assets written to: model0/assets


INFO:tensorflow:Assets written to: model0/assets


211/211 [==============================] - 17s 47ms/step - loss: 0.0365


INFO:tensorflow:Assets written to: model1/assets


...

INFO:tensorflow:Assets written to: model3/assets


INFO:tensorflow:Assets written to: model3/assets


211/211 [==============================] - 17s 42ms/step - loss: 0.2085


INFO:tensorflow:Assets written to: model4/assets


INFO:tensorflow:Assets written to: model4/assets


## Evaluate one model performances

In [ ]:
y_pred = model(X_test)
mean_squared_error(y_test.to_numpy(), y_pred.numpy().reshape(-1))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0019769398>

# Save model

In [30]:
# model.save('model')

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets
